In [3]:
from ipywidgets import interact, IntRangeSlider, Layout, IntSlider
from ipywidgets import Button, Output
from ipywidgets import Text
from ipywidgets import interactive, VBox, HBox
import io
import matplotlib.pyplot as plt
from IPython.display import clear_output
import cv2
import os
import numpy as np
from ipywidgets import FileUpload, interact_manual
%matplotlib inline

In [5]:

def find_image_path(root, image_name):
    for root, _, files in os.walk(root):
        for file in files:
            if file == image_name:
                return os.path.join(root, file)
    return None

def save_mask(mask_rgb, settings, path_img):
    # Save the mask_rgb image
    mask_path = os.path.splitext(path_img)[0] + "_mask.png"
    mask_bgr = cv2.cvtColor(mask_rgb, cv2.COLOR_RGB2BGR)
    cv2.imwrite(mask_path, mask_bgr)
    
    # Save the settings
    settings_path = os.path.splitext(path_img)[0] + "_settings.txt"
    with open(settings_path, 'w') as f:
        f.write(str(settings))
        
def on_slider_change(change):
    #print('on_slider_change called')
    if global_img is not None:  # Only process the image if one has been uploaded
        process_image()

def on_file_upload_change(change):
    #print('on_file_upload_change called')
    if change['type'] == 'change' and change['name'] == 'value':
        global global_img
        file_info = change['new']
        global_img = file_info
        process_image()
    
# Save button
def on_save_button_click(b):
    image_path = find_image_path(root_directory.value, name)
    if image_path is None:
        print(f"Image '{name}' not found in root directory '{root_directory.value}'.")
    else:
        save_mask(mask_rgb, settings, image_path)
save_button = Button(description="Save Mask and Settings")
save_button.on_click(on_save_button_click)

def on_toggle_view_button_click(b):
    global show_mask
    show_mask = not show_mask
    process_image()
toggle_view_button = Button(description="Toggle View (Mask/Overlay)")
toggle_view_button.on_click(on_toggle_view_button_click)

# Your widgets here...
root_directory = Text(description='Root directory:', value='', layout=Layout(width='500px'))
r_range = IntRangeSlider(min=0, max=255, step=1, description='Red', value=(90,120), layout=Layout(width='500px'))
g_range = IntRangeSlider(min=0, max=255, step=1, description='Green', value=(80,110), layout=Layout(width='500px'))
b_range = IntRangeSlider(min=0, max=255, step=1, description='Blue', value=(50,80), layout=Layout(width='500px'))
circle_x = IntSlider(min=0, max=2600, step=1, description='Circle X', value=1300, layout=Layout(width='500px'))
circle_y = IntSlider(min=0, max=2128, step=1, description='Circle Y', value=1064, layout=Layout(width='500px'))
circle_radius = IntSlider(min=500, max=900, step=1, description='Circle Radius', value=685, layout=Layout(width='500px'))
file_upload = FileUpload(accept='.png', multiple=False)
plot_output = Output()

def process_image():
    global name, img, mask_rgb, settings, file_upload, show_mask
    
    #print('process_image called')
    #print('file_upload.value:', file_upload.value)
    
    if not file_upload.value:
        print("Please upload an image.")
        return

    # Use widget values
    r_range_value = r_range.value
    g_range_value = g_range.value
    b_range_value = b_range.value
    circle_x_value = circle_x.value
    circle_y_value = circle_y.value
    circle_radius_value = circle_radius.value
    
    # Change the loop for name, file_info to process the last uploaded file
    name = global_img[0]['name'] 
    img_array = np.frombuffer(global_img[0]['content'] , np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    
    settings = {
    'r_range': r_range_value,
    'g_range': g_range_value,
    'b_range': b_range_value,
    'circle_x': circle_x_value,
    'circle_y': circle_y_value,
    'circle_radius': circle_radius_value
    }
    
    mask_color=(255, 0, 255)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype('uint8')
    
    # Create a black image with the same dimensions as the input image
    circle_mask = np.zeros_like(img).astype('uint8')
    # Draw a white circle on the black image
    cv2.circle(circle_mask, (circle_x_value, circle_y_value), circle_radius_value, (255, 255, 255), -1)
    masked_img = cv2.bitwise_and(img, circle_mask)

    # Threshold color channels
    mask = cv2.inRange(masked_img, (r_range_value[0], g_range_value[0], b_range_value[0]), 
                       (r_range_value[1], g_range_value[1], b_range_value[1]))
    # Replicate mask to 3 channels
    mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    mask_rgb = np.where(mask_rgb == 255, mask_color, mask_rgb).astype('uint8')
    
    # Overlay the colored mask on the original image using cv2.addWeighted
    overlay = cv2.addWeighted(masked_img, 1, mask_rgb, 1, 0)
    
    # Draw the circle on the overlay image
    circle_color = (0, 255, 0)  # Green color for the circle
    cv2.circle(overlay, (circle_x_value, circle_y_value), circle_radius_value, circle_color, 2)
    
    overlay = cv2.addWeighted(overlay, 1, img, .25, 0)
    
    # Visualize the process
    fig, (ax) = plt.subplots(1,1, figsize=(10,10))

    if show_mask:
        ax.imshow(mask_rgb)
    else:
        ax.imshow(overlay)

    ax.axis('off')
    
    with plot_output:
        clear_output(wait=True)  # Clear the previous plot
        plt.show()
        
r_range.observe(on_slider_change, names='value')
g_range.observe(on_slider_change, names='value')
b_range.observe(on_slider_change, names='value')
circle_x.observe(on_slider_change, names='value')
circle_y.observe(on_slider_change, names='value')
circle_radius.observe(on_slider_change, names='value')

file_upload.observe(on_file_upload_change, names='value')

show_mask = False
# Group buttons together in one row
button_row = HBox([file_upload, save_button, toggle_view_button])

display(VBox([root_directory, button_row, r_range, g_range, b_range, circle_x, circle_y, circle_radius, plot_output]))



































Empty space to make the plot better :)